# 决策树

TODO:每轮迭代需要展示的内容：\
1、当前数据集（经过上层选取的特征过滤后的剩余数据），以及总信息熵\
2、计算每个剩余特征的信息熵及信息增益\
3、当前树图\
4、取信息增益最大的作为分类节点\
5、分类后的树图\
[参考https://blog.csdn.net/weixin_43216017/article/details/87474045](https://blog.csdn.net/weixin_43216017/article/details/87474045)

优点：计算复杂度不高，输出结果易于理解，对中间值的缺失不敏感，可以处理不相关特征数据。\
缺点：可能会产生过度匹配问题。\
适用数据类型：数值型和标称型

### 决策树的一般流程

收集数据：可以适用任何方法。\
准备数据：树构造算法只适用于标称型数据，因此数值型数据必须离散化。\
分析数据：可以使用任何方法，构造树完成之后，我们应该检查图形是否符合预期。\
训练算法：构造树的数据结构。\
测试算法：使用经验树计算错误率。\
使用算法：更好地理解数据内在含义。

In [7]:
import os
from collections import Counter

import ipywidgets as widgets
import pandas as pd

from IPython.display import display,Latex
from matplotlib import pyplot as plt

from dataset_producer import Dataset

import trees


In [8]:
DATA_DIR = ".\dataset"
all_data_path = []
for root, dirs, files in os.walk(DATA_DIR):
    for file in files:
        all_data_path.append(os.path.join(root, file))

new_dataset=Dataset()
def show_data(path):
    global new_dataset
    new_dataset=Dataset(path=path,haveHead=True)
    display(new_dataset.data)
widgets.interact(show_data,path=all_data_path)

interactive(children=(Dropdown(description='path', options=('.\\dataset\\human.CSV', '.\\dataset\\lenses.txt',…

<function __main__.show_data(path)>

计算总信息熵

In [44]:
myDat=new_dataset.data.to_numpy()
total_entropy=trees.calcShannonEnt(myDat)

d=Counter(new_dataset.data.iloc[:,-1])
print(new_dataset.head[-1],d)
# fomula='$info(D)=-\\frac{9}{15}log_2(1)$'
fomula='$info(D)='
#每个label产生一个多项式
for i in d:
    term='-\\frac{%d}{%d}log_2(\\frac{%d}{%d})' %(d[i],new_dataset.size,d[i],new_dataset.size)
    fomula+=term
fomula+='=%f$'%total_entropy
# print(fomula)
Latex(fomula)

贷款审批结果 Counter({'是': 9, '否': 6})


<IPython.core.display.Latex object>

计算条件信息熵和信息增益

In [43]:
for i in new_dataset.data.iterrows():
    print(i)
# print(a)
def compute_property(property:str):
    show_info,bestFeat = trees.chooseBestFeatureToSplit(myDat)
    print(show_info)
compute_property(new_dataset.head[0])
widgets.interact(compute_property,property=new_dataset.head[:-1])

(0, 年龄        老年
是否工作       否
是否有房子      是
信贷情况      很好
贷款审批结果     是
Name: 0, dtype: object)
(1, 年龄        老年
是否工作       否
是否有房子      是
信贷情况      很好
贷款审批结果     是
Name: 1, dtype: object)
(2, 年龄        老年
是否工作       是
是否有房子      否
信贷情况       好
贷款审批结果     是
Name: 2, dtype: object)
(3, 年龄        老年
是否工作       是
是否有房子      否
信贷情况       好
贷款审批结果     是
Name: 3, dtype: object)
(4, 年龄        老年
是否工作       否
是否有房子      否
信贷情况      一般
贷款审批结果     否
Name: 4, dtype: object)
(5, 年龄        中年
是否工作       否
是否有房子      否
信贷情况      一般
贷款审批结果     否
Name: 5, dtype: object)
(6, 年龄        中年
是否工作       否
是否有房子      否
信贷情况       好
贷款审批结果     否
Name: 6, dtype: object)
(7, 年龄        中年
是否工作       是
是否有房子      是
信贷情况       好
贷款审批结果     是
Name: 7, dtype: object)
(8, 年龄        中年
是否工作       否
是否有房子      是
信贷情况      很好
贷款审批结果     是
Name: 8, dtype: object)
(9, 年龄        中年
是否工作       否
是否有房子      是
信贷情况      很好
贷款审批结果     是
Name: 9, dtype: object)
(10, 年龄        青年
是否工作       否
是否有房子      否
信贷情况      一般
贷款审批结果     否


In [5]:
def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]#stop splitting when all of the classes are equal
    if len(dataSet[0]) == 1: #stop splitting when there are no more features in dataSet
        return trees.majorityCnt(classList)
    show_info,bestFeat = trees.chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]       #copy all of labels, so trees don't mess up existing labels
        myTree[bestFeatLabel][value] = createTree(trees.splitDataSet(dataSet, bestFeat, value),subLabels)
    return myTree

In [6]:
datas=new_dataset.data.values.tolist()
labels=new_dataset.head[:-1]
myTree  = trees.createTree(datas,labels)
myTree

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\DELL\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-22253ab382a4>", line 3, in <module>
    myTree  = trees.createTree(datas,labels)
  File "C:\Users\DELL\PycharmProjects\mywebsite\tree\trees.py", line 100, in createTree
    show_info,bestFeat = chooseBestFeatureToSplit(dataSet)
  File "C:\Users\DELL\PycharmProjects\mywebsite\tree\trees.py", line 60, in chooseBestFeatureToSplit
    total_d,baseEntropy = calcShannonEnt(dataSet)
TypeError: cannot unpack non-iterable float object

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\DELL\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of 

TypeError: cannot unpack non-iterable float object

使用matplotlib来绘制树形图

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']
trees.createPlot(myTree)